# Análisis de co-ocurrencias y reglas de recomendación por categorías

Este notebook desarrolla un **análisis de co-ocurrencia de categorías a nivel ticket**, con el objetivo de identificar **reglas de recomendación defendibles** (cross-sell) y estimar su **impacto económico potencial**.

A diferencia del recomendador de tallas (orientado a reducción de devoluciones), este análisis se centra en **incremento de valor por pedido**, utilizando patrones reales de compra.

---

## 1. Datos y preparación

Se parte de un dataset a nivel item (`items_venta_cooc.csv`) que contiene:

- identificadores de ticket e item,
- fecha de compra,
- canal (online / físico),
- categoría de producto,
- precio neto unitario,
- margen unitario real.

Para garantizar consistencia:

- se normalizan las categorías (minúsculas, sin tildes),
- se construye una clave temporal `ym` (año-mes),
- se validan fechas y canales.

El universo final cubre más de **900.000 items**, con ventas desde **2017 hasta 2025**, en canal online y físico.

---

## 2. KPIs base de comportamiento de compra

Antes de analizar reglas, se calculan **KPIs estructurales a nivel ticket**, que sirven de contexto:

- número de tickets,
- items por ticket,
- AOV,
- margen medio por ticket,
- porcentaje de tickets multi-item,
- porcentaje de tickets multi-categoría.

Estos KPIs se calculan:

- a nivel global,
- por canal,
- por mes.

Esto permite entender **cuánto margen existe realmente para cross-sell** y evita interpretar reglas fuera de contexto.

---

## 3. Definición de reglas de co-ocurrencia

Se definen reglas explícitas del tipo:

> *“Si el ticket contiene A, recomendar B”*

Ejemplos:
- abrigo → bufanda  
- pantalón → cinturón  
- sudadera → camiseta  

Estas reglas se evalúan **tal como se usarían en tienda**, no como asociaciones arbitrarias.

---

## 4. Métricas de co-ocurrencia

Para cada regla se construyen sets de categorías por ticket y se calculan métricas estándar:

- **support**: frecuencia conjunta A+B,
- **confidence**: probabilidad de B dado A,
- **lift**,
- **lift_vs_sinA**: comparación contra tickets sin A,
- **diff_pp**: mejora absoluta en puntos porcentuales.

La métrica clave es `lift_vs_sinA`, ya que mide si la regla **realmente añade información** frente al comportamiento base.

---

## 5. Estimación de impacto económico

A partir de las métricas de co-ocurrencia, se estima impacto económico bajo distintos escenarios:

- conservador,
- medio,
- agresivo.

Cada escenario se define por:
- *take rate*: % de tickets con A donde la recomendación genera un añadido,
- *canibalización*: fracción de añadidos no incrementales.

El impacto se estima como:

- unidades añadidas esperadas,
- ventas incrementales (€),
- margen incremental (€).

Este enfoque evita asumir conversión total y produce **estimaciones realistas y defendibles**.

---

## 6. Desglose por mes y canal

Las reglas se evalúan también por:

- año-mes,
- canal.

Se aplica un umbral mínimo de tickets para estabilidad estadística.

Esto permite:
- detectar reglas que funcionan solo en ciertos contextos,
- priorizar reglas por impacto real y no por medias globales,
- entender diferencias entre canal online y físico.

Se generan rankings por:
- mayor lift_vs_sinA,
- mayor margen incremental estimado.

---

## 7. Robustez y estabilidad de reglas

Para evitar reglas espurias, se identifican **reglas positivas estables**, imponiendo criterios estrictos:

- lift_vs_sinA > 1,
- diff_pp positivo,
- volumen mínimo de tickets,
- repetición en múltiples grupos (ym + canal).

Este filtrado permite quedarse solo con reglas que:
- funcionan de forma consistente,
- son explicables,
- son aptas para producción.

---

## 8. Mix de negocio por categoría

Como cierre del análisis, se calcula el mix global por categoría:

- % de items,
- % de ventas,
- % de margen.

Esto permite contextualizar las reglas:
- una regla puede ser estadísticamente buena pero poco relevante en margen,
- o tener bajo lift pero alto impacto económico.

---

## 9. Outputs y uso en BI

El notebook exporta tablas listas para análisis y visualización:

- KPIs por canal y por mes,
- métricas globales de co-ocurrencia,
- impacto económico estimado por regla,
- resultados por mes y canal,
- mix global por categoría,
- reglas positivas estables.

Estas tablas se utilizan directamente en Power BI para:
- priorización de reglas,
- estimación de impacto,
- storytelling de negocio.

---

## 10. Conclusión

Este análisis no busca “descubrir asociaciones”, sino **validar reglas accionables** con datos reales.

Las conclusiones principales son:

- no todas las reglas intuitivas aportan valor,
- el impacto se concentra en pocas combinaciones y contextos,
- el valor real está en combinar métricas estadísticas con margen.

El resultado es un conjunto reducido de reglas de cross-sell **defendibles, medibles y alineadas con negocio**, complementarias al recomendador de tallas y al modelo de devoluciones.


In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# =========================
# CONFIG
# =========================
ITEMS_PATH = Path("data/items_venta_cooc.csv")  # tu dataset final
CATALOG_PATH = Path("data/productos.csv")       # opcional (validación)

TICKET_COL = "ticket_id"
DATE_COL   = "fecha_item"
CAT_COL    = "categoría"
CANAL_COL  = "canal"
MARGIN_COL = "margen_unit"
NET_COL    = "precio_neto_unit"

# Reglas que quieres usar en tienda (en minúsculas y sin acentos)
RULES = [
    ("abrigo", "bufanda"),
    ("pantalon", "cinturon"),
    ("calzado", "calcetines"),
    ("sudadera", "camiseta"),
    ("camisa", "cinturon"),
]

# Escenarios para estimar impacto (€)
# - "take_rate": % de tickets con A donde el usuario añade B por la recomendación (incremental)
# - "cannibal": fracción de esos añadidos que NO son incrementales (ya iban a comprar B igualmente)
SCENARIOS = {
    "conservador": {"take_rate": 0.003, "cannibal": 0.50},  # 0.3% con 50% canibalización
    "medio":       {"take_rate": 0.008, "cannibal": 0.35},  # 0.8% con 35% canibalización
    "agresivo":    {"take_rate": 0.015, "cannibal": 0.25},  # 1.5% con 25% canibalización
}

np.random.seed(42)

# =========================
# UTIL: normalización categoría
# =========================
def norm_cat(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.lower()
         .str.normalize("NFD")
         .str.replace(r"[\u0300-\u036f]", "", regex=True)
    )

def safe_dt(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")

# =========================
# CARGA
# =========================
df = pd.read_csv(ITEMS_PATH, low_memory=False)

# columnas auxiliares
df["ym"] = safe_dt(df[DATE_COL]).dt.strftime("%Y%m")
df["_cat"] = norm_cat(df[CAT_COL])
df[CANAL_COL] = df[CANAL_COL].astype(str).str.strip().str.lower()

# checks básicos
if df["ym"].isna().any():
    bad = df[df["ym"].isna()].head(5)
    raise ValueError(f"Hay fechas inválidas en {DATE_COL}. Ejemplos:\n{bad[[DATE_COL,TICKET_COL]].to_string(index=False)}")

print(f"Items: {ITEMS_PATH} | filas={len(df):,} cols={df.shape[1]}")
print("Canales:", sorted(df[CANAL_COL].unique().tolist()))
print("YM range:", df["ym"].min(), "→", df["ym"].max())

# =========================
# KPI BASE (GLOBAL + por canal + por ym)
# =========================
print("\n[KPI] Calculando KPIs base (tickets, AOV, margen, items/ticket, % multi-item)...")

# ticket-level agregados
ticket_base = (
    df.groupby([TICKET_COL], as_index=False)
      .agg(
          canal=(CANAL_COL, "first"),
          ym=("ym", "first"),
          items=("item_id", "count"),
          ventas=(NET_COL, "sum"),
          margen=(MARGIN_COL, "sum"),
          n_cats=("_cat", "nunique"),
      )
)
ticket_base["is_multi_item"] = (ticket_base["items"] > 1).astype(int)

def kpi_table(g: pd.DataFrame) -> pd.Series:
    return pd.Series({
        "tickets": len(g),
        "items_por_ticket": g["items"].mean(),
        "AOV_eur": g["ventas"].mean(),
        "margen_por_ticket_eur": g["margen"].mean(),
        "margen_total_eur": g["margen"].sum(),
        "%_multi_item": g["is_multi_item"].mean() * 100,
        "%_multi_cat": (g["n_cats"] > 1).mean() * 100,
    })

kpi_global = kpi_table(ticket_base)
kpi_canal = ticket_base.groupby("canal").apply(kpi_table).reset_index()
kpi_ym = ticket_base.groupby("ym").apply(kpi_table).reset_index()

print("\n=== KPI GLOBAL ===")
print(kpi_global.to_string())
print("\n=== KPI por canal ===")
print(kpi_canal.to_string(index=False))
print("\n=== KPI por mes (primeros 12) ===")
print(kpi_ym.head(12).to_string(index=False))

# =========================
# COOC METRICS por regla
# =========================
print("\n[COOC] Construyendo sets por ticket (para métricas de co-ocurrencia)...")

# set de categorías por ticket
ticket_cats = (
    df.groupby([TICKET_COL])["_cat"]
      .apply(lambda x: set(x.values))
)

# para desglose por ym+canal: índices de tickets por grupo
ticket_group = (
    ticket_base[[TICKET_COL, "ym", "canal"]]
    .set_index(TICKET_COL)
)

def cooc_metrics_from_sets(sets: pd.Series, A: str, B: str) -> dict:
    # sets: Series[ticket_id] -> set(cats)
    N = len(sets)
    hasA = sets.apply(lambda s: A in s)
    hasB = sets.apply(lambda s: B in s)
    nA = int(hasA.sum())
    nB = int(hasB.sum())
    nAB = int((hasA & hasB).sum())

    support = nAB / N if N else np.nan
    conf = nAB / nA if nA else np.nan
    pB = nB / N if N else np.nan
    pB_notA = ( ( (~hasA) & hasB ).sum() / ( (~hasA).sum() ) ) if ((~hasA).sum() > 0) else np.nan

    lift = (conf / pB) if (pB and pB > 0 and conf==conf) else np.nan
    lift_vs_sinA = (conf / pB_notA) if (pB_notA and pB_notA > 0 and conf==conf) else np.nan
    diff_pp = (conf - pB_notA) * 100 if (pB_notA==pB_notA and conf==conf) else np.nan

    return {
        "tickets": N,
        "count_A": nA,
        "count_B": nB,
        "count_AB": nAB,
        "support_AB": support,
        "confidence": conf,
        "pB": pB,
        "pB_sinA": pB_notA,
        "lift": lift,
        "lift_vs_sinA": lift_vs_sinA,
        "diff_pp": diff_pp,
        "A": A,
        "B": B,
    }

# margen medio del producto B (real, desde tus items)
# OJO: esto es margen unitario medio del B en el dataset (con descuentos reales)
margin_by_cat = df.groupby("_cat")[MARGIN_COL].mean().to_dict()
price_by_cat = df.groupby("_cat")[NET_COL].mean().to_dict()

# calcular métricas globales por regla
rules_global = []
for A, B in RULES:
    rules_global.append(cooc_metrics_from_sets(ticket_cats, A, B))
rules_global_df = pd.DataFrame(rules_global)

print("\n=== COOC GLOBAL (tus reglas) ===")
cols_show = ["A","B","tickets","count_A","count_B","count_AB","support_AB","confidence","lift","lift_vs_sinA","diff_pp"]
print(rules_global_df[cols_show].sort_values("lift_vs_sinA", ascending=False).to_string(index=False))

# =========================
# € POTENCIAL por regla (escenarios)
# =========================
print("\n[€] Estimando impacto € potencial (escenarios)...")

def estimate_euros(row: pd.Series, scenario: dict) -> dict:
    # base: en tickets con A, añadimos B incrementalmente a take_rate.
    # quitamos canibalización (parte que ya iba a pasar).
    take = scenario["take_rate"]
    cann = scenario["cannibal"]
    nA = row["count_A"]

    # nº añadidos incrementales esperados
    adds = nA * take * (1 - cann)

    # valor por añadido: margen medio de B (y opcional: ventas)
    B = row["B"]
    m_unit = float(margin_by_cat.get(B, np.nan))
    v_unit = float(price_by_cat.get(B, np.nan))

    return {
        "adds_est": adds,
        "inc_margin_eur": adds * m_unit if m_unit == m_unit else np.nan,
        "inc_sales_eur": adds * v_unit if v_unit == v_unit else np.nan,
    }

rows = []
for _, r in rules_global_df.iterrows():
    base = {
        "A": r["A"],
        "B": r["B"],
        "count_A": r["count_A"],
        "confidence": r["confidence"],
        "lift_vs_sinA": r["lift_vs_sinA"],
        "diff_pp": r["diff_pp"],
        "margen_unit_B_avg": margin_by_cat.get(r["B"], np.nan),
        "precio_net_unit_B_avg": price_by_cat.get(r["B"], np.nan),
    }
    for name, sc in SCENARIOS.items():
        est = estimate_euros(r, sc)
        rows.append({**base, "scenario": name, **est})

impact_df = pd.DataFrame(rows)
impact_df = impact_df.sort_values(["scenario","inc_margin_eur"], ascending=[True, False])

print("\n=== IMPACTO € ESTIMADO (GLOBAL) ===")
print(
    impact_df[["scenario","A","B","adds_est","inc_sales_eur","inc_margin_eur","margen_unit_B_avg","lift_vs_sinA","diff_pp"]]
    .to_string(index=False)
)

# =========================
# DESGLOSE por YM + CANAL (ranking)
# =========================
print("\n[COOC] Desglose por YM+CANAL (ranking por lift_vs_sinA y por € estimado)...")

# pre-agrupamos tickets por (ym, canal)
group_to_tickets = (
    ticket_group.reset_index()
              .groupby(["ym","canal"])[TICKET_COL]
              .apply(list)
              .to_dict()
)

# para no recalcular sets globales, creamos un df con sets + join
sets_df = ticket_cats.to_frame("cats").join(ticket_group)

out = []
keys = list(group_to_tickets.keys())
for (ym, canal) in tqdm(keys, desc="Grupos ym+canal"):
    sub_sets = sets_df[(sets_df["ym"] == ym) & (sets_df["canal"] == canal)]["cats"]
    if len(sub_sets) < 500:   # umbral mínimo para estabilidad (ajusta si quieres)
        continue
    for A, B in RULES:
        met = cooc_metrics_from_sets(sub_sets, A, B)
        met["ym"] = ym
        met["canal"] = canal

        # impacto (escenario medio) en ese grupo
        est = estimate_euros(pd.Series(met), SCENARIOS["medio"])
        met.update({f"adds_est_medio": est["adds_est"], f"inc_margin_medio": est["inc_margin_eur"]})
        out.append(met)

by_group_df = pd.DataFrame(out)

# top por lift_vs_sinA
top_lift = (
    by_group_df.sort_values(["lift_vs_sinA","diff_pp"], ascending=False)
               .head(30)[["ym","canal","A","B","tickets","count_A","count_AB","confidence","lift_vs_sinA","diff_pp"]]
)
print("\n=== TOP 30 por lift_vs_sinA (YM+canal) ===")
print(top_lift.to_string(index=False))

# top por margen incremental (escenario medio)
top_margin = (
    by_group_df.sort_values(["inc_margin_medio","lift_vs_sinA"], ascending=False)
               .head(30)[["ym","canal","A","B","tickets","count_A","adds_est_medio","inc_margin_medio","lift_vs_sinA","diff_pp"]]
)
print("\n=== TOP 30 por € margen incremental (escenario medio) ===")
print(top_margin.to_string(index=False))

# =========================
# REPARTO TOTAL por categoría (% items y % margen)
# =========================
print("\n[MIX] Reparto total por categoría (% items y % margen)...")

mix = (
    df.groupby("_cat")
      .agg(
          items=("item_id","count"),
          ventas=(NET_COL,"sum"),
          margen=(MARGIN_COL,"sum"),
          margen_unit_avg=(MARGIN_COL,"mean"),
      )
      .sort_values("items", ascending=False)
)

mix["pct_items"] = mix["items"] / mix["items"].sum() * 100
mix["pct_margen"] = mix["margen"] / mix["margen"].sum() * 100
mix = mix.reset_index().rename(columns={"_cat":"categoria_norm"})

print(mix.to_string(index=False))

# =========================
# EXPORTS para Power BI / README
# =========================
out_dir = Path("data/kpi_recos")
out_dir.mkdir(parents=True, exist_ok=True)

kpi_canal.to_csv(out_dir / "kpi_por_canal.csv", index=False)
kpi_ym.to_csv(out_dir / "kpi_por_ym.csv", index=False)
rules_global_df.to_csv(out_dir / "cooc_global_reglas.csv", index=False)
impact_df.to_csv(out_dir / "impacto_estimado_global.csv", index=False)
by_group_df.to_csv(out_dir / "cooc_por_ym_canal.csv", index=False)
mix.to_csv(out_dir / "mix_global_por_categoria.csv", index=False)

print(f"\nOK: exports en {out_dir.as_posix()}")
print(" - kpi_por_canal.csv")
print(" - kpi_por_ym.csv")
print(" - cooc_global_reglas.csv")
print(" - impacto_estimado_global.csv")
print(" - cooc_por_ym_canal.csv")
print(" - mix_global_por_categoria.csv")


Items: data\items_venta_cooc.csv | filas=905,445 cols=21
Canales: ['fisico', 'online']
YM range: 201708 → 202509

[KPI] Calculando KPIs base (tickets, AOV, margen, items/ticket, % multi-item)...


C:\Users\PEDRO\AppData\Local\Temp\ipykernel_8988\3588324404.py:106: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_canal = ticket_base.groupby("canal").apply(kpi_table).reset_index()
C:\Users\PEDRO\AppData\Local\Temp\ipykernel_8988\3588324404.py:107: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  kpi_ym = ticket_base.groupby("ym").apply(kpi_table).reset_index()



=== KPI GLOBAL ===
tickets                  5.840180e+05
items_por_ticket         1.550372e+00
AOV_eur                  8.825922e+01
margen_por_ticket_eur    4.594730e+01
margen_total_eur         2.683405e+07
%_multi_item             4.593780e+01
%_multi_cat              3.707300e+01

=== KPI por canal ===
 canal  tickets  items_por_ticket   AOV_eur  margen_por_ticket_eur  margen_total_eur  %_multi_item  %_multi_cat
fisico 130000.0          1.699108 97.742382              50.589013        6576571.70     48.360000    40.856923
online 454018.0          1.507784 85.543883              44.618226       20257477.85     45.244241    35.989542

=== KPI por mes (primeros 12) ===
    ym  tickets  items_por_ticket   AOV_eur  margen_por_ticket_eur  margen_total_eur  %_multi_item  %_multi_cat
201708    397.0          1.503778 72.673174              39.857053          15823.25     48.614610    32.745592
201709    519.0          1.489403 73.064451              40.420906          20978.45     47.5915

Grupos ym+canal: 100%|███████████████████████████████████████████████████████████████| 143/143 [00:12<00:00, 11.08it/s]



=== TOP 30 por lift_vs_sinA (YM+canal) ===
    ym  canal        A          B  tickets  count_A  count_AB  confidence  lift_vs_sinA  diff_pp
202403 fisico  calzado calcetines     3136      225         7    0.031111      1.775773 1.359136
202209 fisico   camisa   cinturon     1169      113         3    0.026549      1.752212 1.139716
202212 fisico  calzado calcetines     1545      173         5    0.028902      1.724051 1.213789
202210 fisico   camisa   cinturon     1138      120         7    0.058333      1.649537 2.296988
202204 fisico   camisa   cinturon     1029       47         2    0.042553      1.607201 1.607661
202307 fisico   camisa   cinturon     2711      273         7    0.025641      1.562821 0.923413
202206 fisico  calzado calcetines     1105      199         8    0.040201      1.517588 1.371094
202305 fisico   abrigo    bufanda     2195      751        30    0.039947      1.442077 1.224591
202207 fisico   camisa   cinturon     1356       64         2    0.031250      1.44

In [2]:
import pandas as pd

df = pd.read_csv("data/items_venta_cooc.csv", low_memory=False)
df["ym"] = pd.to_datetime(df["fecha_item"], errors="coerce").dt.strftime("%Y%m")

# normaliza categoría (sin tildes)
df["_cat"] = (
    df["categoría"].astype(str)
      .str.lower()
      .str.normalize("NFD")
      .str.replace(r"[\u0300-\u036f]", "", regex=True)
)

pairs = [
    ("abrigo","bufanda"),
    ("pantalon","cinturon"),
    ("calzado","calcetines"),
    ("camiseta","gorra"),
]

def cooc_metrics(sub, A, B):
    t = sub.groupby("ticket_id")["_cat"].apply(set)
    N = len(t)
    nA  = (t.apply(lambda s: A in s)).sum()
    nB  = (t.apply(lambda s: B in s)).sum()
    nAB = (t.apply(lambda s: (A in s) and (B in s))).sum()
    if N == 0 or nA == 0 or nB == 0:
        return None
    pB = nB / N
    conf = nAB / nA
    lift = conf / pB if pB > 0 else None
    return N, nA, nB, nAB, conf, lift

# ONLINE global
online = df[df["canal"].astype(str).str.lower().str.strip() == "online"]
print("ONLINE tickets:", online["ticket_id"].nunique())

for A,B in pairs:
    m = cooc_metrics(online, A, B)
    print(A,"->",B, m)


ONLINE tickets: 454018
abrigo -> bufanda (454018, np.int64(152547), np.int64(6516), np.int64(1360), np.float64(0.00891528512524009), np.float64(0.6211939720674118))
pantalon -> cinturon (454018, np.int64(134582), np.int64(10793), np.int64(1740), np.float64(0.012928920658037479), np.float64(0.5438675715112443))
calzado -> calcetines (454018, np.int64(29132), np.int64(5878), np.int64(315), np.float64(0.010812851846766442), np.float64(0.8351870312632199))
camiseta -> gorra (454018, np.int64(174550), np.int64(11752), np.int64(2560), np.float64(0.014666284732168433), np.float64(0.5666063020362192))


In [3]:
import pandas as pd
import numpy as np

by_group_df = pd.read_csv("data/kpi_recos/cooc_por_ym_canal.csv")

# filtros "defendibles"
POS_LIFT = 1.10
POS_DIFFPP = 0.50
MIN_TICKETS = 800

pos = by_group_df[
    (by_group_df["tickets"] >= MIN_TICKETS) &
    (by_group_df["lift_vs_sinA"] >= POS_LIFT) &
    (by_group_df["diff_pp"] >= POS_DIFFPP)
].copy()

print("Grupos positivos:", len(pos), "de", len(by_group_df))

# ranking por regla (cuántas veces funciona)
summary = (
    pos.groupby(["A","B"])
       .agg(
           grupos=("tickets","count"),
           tickets_tot=("tickets","sum"),
           lift_med=("lift_vs_sinA","mean"),
           diffpp_med=("diff_pp","mean"),
           conf_med=("confidence","mean"),
           support_med=("support_AB","mean"),
       )
       .sort_values(["grupos","lift_med"], ascending=False)
       .reset_index()
)

print("\n=== Reglas más estables (solo donde mejora de verdad) ===")
print(summary.to_string(index=False))

# si quieres: export para Power BI
out = "data/kpi_recos/reglas_positivas_estables.csv"
summary.to_csv(out, index=False)
print("\nOK:", out)


Grupos positivos: 13 de 705

=== Reglas más estables (solo donde mejora de verdad) ===
      A          B  grupos  tickets_tot  lift_med  diffpp_med  conf_med  support_med
 camisa   cinturon       7        10104  1.505372    1.184542  0.035970     0.002983
calzado calcetines       5        14418  1.555603    1.013386  0.028304     0.003339
 abrigo    bufanda       1         2195  1.442077    1.224591  0.039947     0.013667

OK: data/kpi_recos/reglas_positivas_estables.csv
